In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno  
import datatable as dt
import re
from sklearn import metrics

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [2]:
train = pd.read_csv('train.csv')
train.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [3]:
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.')
train.replace(['Capt','Col','Countess','Don','Dr','Jonkheer','Lady','Major','Rev','Sir','Dona'],'Others',inplace=True)
train.replace(['Mlle','Ms'],'Miss',inplace = True)
train.replace('Mme','Mrs',inplace=True)
train.Title.replace({'Mr':0,'Miss':1,'Mrs':2,'Master':3,'Others':4},inplace=True)
# ------------------------------------------------------------------------------
train['Fare'] = pd.qcut(train.Fare,4,labels=[0,1,2,3])
train.drop(['PassengerId','Name','Ticket','Cabin'],axis = 1, inplace = True)
train.Sex.replace({'male':0,'female':1}, inplace=True)
train.Embarked.replace({'S':0,'C':1,'Q':2,np.nan:0},inplace=True)
train.Age.fillna(28.0,inplace=True)
train.Age = train.Age.astype(int)

In [4]:
def cond(s):
    if (s.Sex==1) or (s.Pclass==1) or (s.child==1):
        return 1
    else: 
        return 0

In [5]:
def cond_2(s):
    if (s.SibSp == 0) and (s.Parch == 0):
        return 1
    else:
        return 0

In [6]:
train['child'] = np.where(train.Age < 17,1,0)
train['Priority'] = train.apply(cond,axis=1)
train['Alone'] = train.apply(cond_2,axis=1)

In [14]:
train.Priority.value_counts()

1.0    484
0.0    407
Name: Priority, dtype: int64

In [7]:
test = pd.read_csv('test.csv')
test.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


In [8]:
test['Title'] = test.Name.str.extract(' ([A-Za-z]+)\.')
test.replace(['Capt','Col','Countess','Don','Dr','Jonkheer','Lady','Major','Rev','Sir','Dona'],'Others',inplace=True)
test.replace(['Mlle','Ms'],'Miss',inplace = True)
test.replace('Mme','Mrs',inplace=True)
test.Title.replace({'Mr':0,'Miss':1,'Mrs':2,'Master':3,'Others':4},inplace=True)
# ------------------------------------------------------------------------------
test.Fare.fillna(test.Fare.median(),inplace=True)
test['Fare'] = pd.qcut(train.Fare,4,labels=[0,1,2,3])
test.drop(['PassengerId','Name','Ticket','Cabin'],axis = 1, inplace = True)
test.Embarked.replace({'S':0,'C':1,'Q':2,np.nan:0},inplace=True)
test.Sex.replace({'male':0,'female':1}, inplace=True)
test.Age.fillna(28.0,inplace=True)
test.Age = test.Age.astype(int)

In [9]:
test['child'] = np.where(test.Age < 17,1,0)
test['Priority'] = test.apply(cond,axis=1)
test['Alone'] = test.apply(cond_2,axis=1)

In [10]:
scaler = MinMaxScaler()
test_n = scaler.fit_transform(test)
test = pd.DataFrame(test_n, columns = test.columns)

train_n = scaler.fit_transform(train)
train = pd.DataFrame(train_n, columns = train.columns)

In [11]:
X,y = train.drop('Survived',axis=1),train.Survived
lr = LogisticRegression(max_iter=1000)
lr.fit(X,y)
pred = lr.predict(test)

In [12]:
pred = pred.astype(int)

In [13]:
test_t = pd.read_csv('test.csv')
sub = pd.DataFrame({'PassengerId':test_t.PassengerId,'Survived':pred})
sub.to_csv('submission.csv',index=False)
sub.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
